<a href="https://colab.research.google.com/github/chetools/CHE4071_Fall2025/blob/main/ExothermicTubularReactor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.7 MB/s eta 0:00:00


In [2]:
from importnb import Notebook
with Notebook():
    from che5 import sim, pid, TF1, TF2, shift

import numpy as np
import scipy as sp
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "plotly_dark"
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)

from plotly.subplots import make_subplots

from sympy.abc import s
from sympy import exp, Symbol, simplify
import scipy as sp
import scipy.signal as sig

In [3]:
jnp.set_printoptions(precision=3, linewidth=200)

In [181]:
N=50
q=100. #L/min
ca_in = 1. #mol/L
T_in = 350. #K
Vtotal = 100. #L
V = Vtotal/N  #L
rho = 1000. #g/L
C = 0.239 #J/(g K)
negHr = 5e4 #J/mol
ER = 8750. #K
k0 = 7.2e10 #1/min

UAtot = 9e4 #J/(min K)
UA = UAtot/N

Tc_ss = 300.3 #K
Ca_ss_guess =  np.full(N, 0.5) #mol/L
T_ss_guess = np.full(N, 350.) #K
tend = 5. #min

In [182]:
def rhs(t, vec):
    Tc=Tc_ss
    ca, T = jnp.split(vec,2)
    dca, dT = jnp.zeros_like(ca), jnp.zeros_like(T)
    k=k0*jnp.exp(-ER/T)
    dca=dca.at[0].set(q*(ca_in - ca[0])/V - k[0]*ca[0])
    dT=dT.at[0].set((rho*q*C*(T_in-T[0]) - UA*(T[0]-Tc) + negHr*V*k[0]*ca[0])/(rho*V*C))

    dca=dca.at[1:].set(q*(ca[:-1] - ca[1:])/V - k[1:]*ca[1:])
    dT=dT.at[1:].set((rho*q*C*(T[:-1]-T[1:]) - UA*(T[1:]-Tc) + negHr*V*k[1:]*ca[1:])/(rho*V*C))
    return jnp.r_[dca, dT]

rhs=jax.jit(rhs)
rhs_jac = jax.jit(jax.jacobian(rhs, argnums=1))

In [183]:

res=sp.integrate.solve_ivp(rhs, (0, tend), np.r_[Ca_ss_guess, T_ss_guess], method='Radau', dense_output=True, jac=rhs_jac)
res

  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 0.000e+00  1.297e-02 ...  3.922e+00  5.000e+00]
        y: [[ 5.000e-01  7.327e-01 ...  9.798e-01  9.798e-01]
            [ 5.000e-01  5.627e-01 ...  9.593e-01  9.593e-01]
            ...
            [ 3.500e+02  3.489e+02 ...  3.260e+02  3.260e+02]
            [ 3.500e+02  3.489e+02 ...  3.242e+02  3.242e+02]]
      sol: <scipy.integrate._ivp.common.OdeSolution object at 0x7f9bc1492d20>
 t_events: None
 y_events: None
     nfev: 288
     njev: 6
      nlu: 46

In [184]:
tplot=np.linspace(0,tend,20)
cplot, Tplot = np.split(res.sol(tplot),2,axis=0)
fig = make_subplots(rows=1, cols=2)
tank = np.arange(N)
for i, t in enumerate(tplot):
    fig.add_scatter(x=tank, y=cplot[:,i], row=1, col=1, name=f'c{t:0.1f}', legendgroup=t)
    fig.add_scatter(x=tank, y=Tplot[:,i], row=1, col=2, name=f'T{t:0.1f}', legendgroup=t)
fig.update_layout(height=600, width=1000)

In [64]:
res.y[:,-1]

array([5.296e-03, 2.522e-04, 6.725e-05, 4.399e-05, 3.854e-05, 4.820e+02, 4.290e+02, 3.910e+02, 3.642e+02, 3.452e+02])

In [65]:
rhs(0,res.y[:,-1])

Array([ 5.400e-12,  3.523e-12,  5.315e-12,  2.249e-11, -3.573e-09, -2.707e-08, -2.776e-07, -1.440e-06, -5.114e-06, -1.444e-05], dtype=float64)

In [66]:
sp.optimize.root(lambda v: rhs(0., v), res.y[:,-1], jac = lambda v: rhs_jac(0., v), method='lm',options=dict(ftol=1e-15) )

 message: The relative error between two consecutive iterates is at most 0.000000
 success: True
  status: 2
     fun: [-8.771e-15 -3.036e-17 -4.879e-19  4.811e-19 -8.677e-17
            1.787e-12 -5.874e-14 -2.611e-14  1.019e-13 -3.329e-14]
       x: [ 5.296e-03  2.522e-04  6.725e-05  4.399e-05  3.854e-05
            4.820e+02  4.290e+02  3.910e+02  3.642e+02  3.452e+02]
   cov_x: [[ 2.439e-05  2.326e-06 ... -5.051e-02 -3.560e-02]
           [ 2.326e-06  8.492e-05 ...  1.316e-01  9.303e-02]
           ...
           [-5.051e-02  1.316e-01 ...  7.515e+02  5.661e+02]
           [-3.560e-02  9.303e-02 ...  5.661e+02  4.443e+02]]
  method: lm
    nfev: 4
    njev: 1
    fjac: [[ 1.965e+05 -2.545e-05 ... -0.000e+00 -0.000e+00]
           [-2.672e-03  2.092e+04 ... -0.000e+00 -0.000e+00]
           ...
           [ 3.210e+01  5.000e+00 ... -3.674e-02  9.971e-01]
           [ 0.000e+00 -1.265e-08 ...  1.448e-02 -2.997e-02]]
    ipvt: [0 1 2 3 4 8 9 6 5 7]
     qtf: [-2.707e-08 -2.776e-07 -1.